In [1]:
import pandas as pd
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.prompts import ChatPromptTemplate
# from llm_client_chat import AlpacaLLM
from llm_client import AlpacaLLM

import time, os
os.environ['HF_HOME'] = './cache/'

In [2]:
def load_embedding_model(model_path : str):
    start_time = time.time()
    encode_kwargs = {"normalize_embeddings": True}
    local_embedding = HuggingFaceEmbeddings(
        model_name=model_path,
        cache_folder="./models",
        encode_kwargs=encode_kwargs
    )
    end_time = time.time()
    print(f'model load time {round(end_time - start_time, 0)} second')
    return local_embedding

embedding = load_embedding_model(model_path="intfloat/multilingual-e5-large")

d:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model load time 149.0 second


In [3]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

In [4]:
excel_pertanyaan = pd.read_excel("Pertanyaan Benchmark.xlsx", sheet_name=None, index_col=0)

In [5]:
# CHROMA_PATH = "final_test/unstructured"
CHROMA_PATH = "final_test/langchain"
# ==================================
embed_model = "Multilingual-e5-large"
# ==================================
# loader = "Unstructured"
loader = "langchain"
# ==================================
reranker = "bge-reranker-v2-m3"

In [6]:
## run without reranking
# db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

# PROMPT_TEMPLATE = """
# Jawab pertanyaan hanya menggunakan informasi di bawah ini, suatu informasi mungkin tidak mengandung informasi yang relevan dengan pertanyaan, masing-masing informasi dipisahkan oleh '---':

# Pertanyaan : {question}
# ---

# {context}

# ---
# """

# for sheet in excel_pertanyaan:
#     jawaban = []
#     generated_response = []
#     for num, pertanyaan in enumerate(excel_pertanyaan[sheet]["Pertanyaan"]):
#         if type(pertanyaan) == str:
#             results = db.similarity_search_with_relevance_scores(pertanyaan, k=3)
#             if len(results) == 0 or results[0][1] < 0.7:
#                 print(f"Similarity too low.", end="\n")

#             context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
#             prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
#             prompt = prompt_template.format(context=context_text, question=pertanyaan)
#             model = AlpacaLLM()
#             response_text = model.invoke(prompt)
#             generated_response.append(response_text)

#             list_jwbn = []
#             for doc, _score in results:
#                 string = (f"{doc.page_content}"
#                 f"\nmetadata: {doc.metadata}"
#                 f"\nscore: {_score}")
#                 list_jwbn.append(string)

#             jawaban.append(list_jwbn)
#         else:
#             jawaban.append([pertanyaan]*3)

#     jawaban1 = [doc[0] for doc in jawaban]
#     jawaban2 = [doc[1] for doc in jawaban]
#     jawaban3 = [doc[2] for doc in jawaban]
#     # jawaban4 = [doc[3] for doc in jawaban]
#     # jawaban5 = [doc[4] for doc in jawaban]

#     excel_pertanyaan[sheet]["Dokumen 1"] = jawaban1
#     excel_pertanyaan[sheet]["Dokumen 2"] = jawaban2
#     excel_pertanyaan[sheet]["Dokumen 3"] = jawaban3
#     excel_pertanyaan[sheet]["Generated Answer"] = generated_response
#     # excel_pertanyaan[sheet]["Dokumen 4"] = jawaban4
#     # excel_pertanyaan[sheet]["Dokumen 5"] = jawaban5

In [7]:
## Run with Reranking
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker

retriever = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding).as_retriever(search_kwargs={"k": 20})
PROMPT_TEMPLATE = """
Jawablah pertanyaan di bawah ini hanya dengan menggunakan potongan teks di bawah ini, 
tidak semua teks relevan dengan pertanyaan yang ditanyakan, 
anda hanya perlu menjawab pertanyaan yang ditanyakan.
Jawablah pertanyaan berikut menggunakan bahasa inggris.
masing-masing teks dipisahkan oleh '---':

Pertanyaan : {question}
---

{context}

---
"""
# lst = ["tatalaksana pada anak"]
for sheet in excel_pertanyaan:
    jawaban = []
    generated_response = []
    for num, pertanyaan in enumerate(excel_pertanyaan[sheet]["Pertanyaan"]):
        if type(pertanyaan) == str:
            compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
            compression_retriever = ContextualCompressionRetriever(
                base_compressor=compressor, base_retriever=retriever
            )
            results = compression_retriever.invoke(pertanyaan)
            
            context_text = "\n\n---\n\n".join([doc.page_content for doc in results])
            prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
            prompt = prompt_template.format(context=context_text, question=pertanyaan)
            model = AlpacaLLM()
            response_text = model.invoke(prompt)
            generated_response.append(response_text)

            list_jwbn = []
            for doc in results:
                string = (f"{doc.page_content}"
                f"\nmetadata: {doc.metadata}")
                list_jwbn.append(string)

            jawaban.append(list_jwbn)
        else:
            jawaban.append([pertanyaan]*3)
            generated_response.append(pertanyaan)

    
    jawaban1 = [doc[0] for doc in jawaban]
    jawaban2 = [doc[1] for doc in jawaban]
    jawaban3 = [doc[2] for doc in jawaban]
    # jawaban4 = [doc[3] for doc in jawaban]
    # jawaban5 = [doc[4] for doc in jawaban]

    excel_pertanyaan[sheet]["Dokumen 1"] = jawaban1
    excel_pertanyaan[sheet]["Dokumen 2"] = jawaban2
    excel_pertanyaan[sheet]["Dokumen 3"] = jawaban3
    excel_pertanyaan[sheet]["Generated Answer"] = generated_response
    # excel_pertanyaan[sheet]["Dokumen 4"] = jawaban4
    # excel_pertanyaan[sheet]["Dokumen 5"] = jawaban5

In [8]:
excel_pertanyaan["PNPK"]

,Pertanyaan,Jawaban,Dokumen,tipe sumber,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa penyebab kerusakan ginjal pada anak?,Penyebab utama penyakit ginjal tahap akhir (PG...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,abnormalitas struktur atau fungsi ginjal denga...,ABSTRAK\nPenyakit ginjal kronik (PGK) merupaka...,Langkah pertama dalam proses deteksi dini PGK ...,"\nTidak hanya itu, pula, penderita PG"
2,Apa yang harus dijelaskan saat melakukan eduka...,1. Penjelasan mengenai perjalanan alamiah peny...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,sebuah senter di Korea dengan melibatkan 1218 ...,"1) Pasien PGK tahap 4 (eGFR <30ml/menit/1,73m2...",2) Persiapan Continuous Ambulatory Peritoneal ...,\nPersiapan dan Inisiasi Pasien Hemodialisis K
3,Apa yang harus dipersiapkan pada pasien/ kelua...,1. Informed consent setelah mendapatkan penjel...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,diberikan cairan sebagai berikut:\n(1)Infus no...,"obat seperti hiperkahiperurisemia, hiperfosfat...","klirens zat terlarut, hal ini harus \ndipertim...",\nyang harus dipersiapkan pada pasien/kel
4,Apa saja komplikasi akut yang dapat terjadi sa...,1. Hipotensi intradialisis\n2. kram otot\n3. g...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,"obat seperti hiperkahiperurisemia, hiperfosfat...",kebocoran darah berbunyi atau didapatkan darah...,a) Perbaikan nutrisi: peningkatan berat badan ...,\nterjadi pada waktu hemodialisis disebab
5,Apa langkah pertama dalam deteksi dini Penyaki...,Langkah pertama dalam proses deteksi dini PGK ...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,Langkah pertama dalam proses deteksi dini PGK ...,"kehilangan konsentrasi, lesu hingga kejang, ko...",PENDAHULUAN\nPenyakit ginjal kronik (PGK) atau...,\nlangkah penting dalam pencegahan dan pengaw
6,Apa syarat untuk memasang kembali kateter hemo...,"tidak demam selama 48-72 jam, kultur darah neg...",Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,Vankomisin intravena (pada infeksi MRSA atau C...,kultur darah negatif 2 kali berturutan; 3) kad...,"infeksi), probable (kultur darah positif oleh ...",\nabdomen untuk mengetahui perubahan organ perit
7,Apa tujuan dari terapi nutrisi perioperatif?,Tujuan terapi nutrisi adalah memperbaiki statu...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,setidaknya satu dari kriteria dibawah ini: \n...,[Peringkat bukti: Level III] \nDari hasil pene...,"yang tidak adekuat, inhibisi pelepasan besi d...",\nTidak ada terlalu banyak informasi tentang
8,Apa saja kriteria yang menunjukkan pasien beda...,Kriteria tersebut meliputi penurunan berat bad...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,f. Status volume cairan Pasien PGK berisiko u...,setidaknya satu dari kriteria dibawah ini: \n...,[Peringkat bukti: Level III] \nDari hasil pene...,\nyang menunjukkan risiko nutrisi berat dal
9,Apa saja efek samping hemodialisis?,"Hipotensi Intradialisis, Kram otot, nyeri dada...",Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,kebocoran darah berbunyi atau didapatkan darah...,"konsekuensi klinis yang merugikan, termasuk \n...",dan pertumbuhan pada anak membutuhkan\npendeka...,\ncabik dan tidak adanya peningkatan konsent


In [9]:
excel_pertanyaan["Nutrisi"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa saja parameter penilaian status nutrisi da...,"Parameter penilaian meliputi asupan makanan, p...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,rekomendasi terapi nutrisi pada anak dengan P...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,Rekomendasi parameter dan frekuensi penilaian...,\n##
2,Apa tujuan terapi nutrisi pada anak dengan PGK?,Tujuan terapi nutrisi pada anak dengan PGK ada...,teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,PENDAHULUAN Penyakit ginjal kronik (PGK) adala...,rekomendasi terapi nutrisi pada anak dengan P...,Question: What is the purpose of nutrition the...
3,apa yang menyebabkan gangguan selera makan pad...,"Pada anak dengan poliuria, rasa haus lebih nya...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,d. Kekurangan berat badan lebih dari 10% berat...,"pada anak dengan PGK derajat 2-5 dan 5D, peraw...",Tujuan terapi nutrisi ini dapat dicapai dengan...,\nPersatuan Pediatri Amerika Serikat (American...
4,bagaimana gambaran penyakit ginjal kronik dera...,Kerusakan ginjal dengan LFG normal atau mening...,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Klasifikasi derajat penyakit ginjal kronik.Der...,PENDAHULUAN Penyakit ginjal kronik (PGK) adala...,beberapa faktor yang menyebabkan kerusakan\nbe...,\n1. Mengontrol gejala utama seperti per
5,berapa interval penilaian asupan makanan anak ...,1 - 3 bulan,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi parameter dan frekuensi penilaian...,Rekomendasi asupan makanan yang adekuat pada ...,rekomendasi terapi nutrisi pada anak dengan P...,\n## Perawatan Rumah Sakit\n\nPenyakit COVID
6,berapa rekomendasi asupan asam pantotenat untu...,4 mg/hr,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi asupan makanan yang adekuat pada ...,Rekomendasi asupan protein pada anak PGK dera...,. Rekomendasi KDOQI untuk asupan fosfat pada a...,\nRekomendasi KDOQI untuk asupan nat
7,apa penyebab kekurangan vitamin D pada anak de...,"Gaya hidup dengan aktivitas fisik rendah, sehi...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,b. Vitamin D Pada anak dengan PGK terdapat def...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,pada anak PGK 3-5 dan 5D. Setelah diet restrik...,\nvitamin D setiap tahunnya.
8,berapa rekomendasi asupan protein untuk pasien...,"0,95-1,95 g/kg/hari",tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi asupan protein pada anak PGK dera...,. Asupan protein untuk anak PGK berdasarkan DR...,penyebab 25% kematian dan kesakitan pada anak...,\n## Perawatan Fisioterapi Dalam Pengobatan
9,apakah pasien anak PGK perlu mengurangi asupan...,Pada anak pengidap PGK derajat 3-5 dengan olig...,teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,bermanfaat mencegah dan menangani hiperpar...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,pembatasan diet natrium juga dapat membantu me...,\npasien PGK dengan oliguria.


In [10]:
excel_pertanyaan["tatalaksana pada anak"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,apa faktor yang menyebabkan kerusakan berkelan...,"glomerulosklerosis, pembentukan fibrosis tubul...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",beberapa faktor yang menyebabkan kerusakan\nbe...,abnormalitas struktur atau fungsi ginjal denga...,menunjukkan hematuria dan proteinuria pada ana...,\nhiperfosfatemia dan asidosis akibat penur
2,apa penyebab umum penyakit ginjal kronis pada ...,abnormalitas kongenital seperti hipoplasia ata...,teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",abnormalitas struktur atau fungsi ginjal denga...,ABSTRAK\nPenyakit ginjal kronik (PGK) merupaka...,Tujuan terapi nutrisi ini dapat dicapai dengan...,\nPenyakit Ginjal Kronis (PGK) mer
3,apa peran ACE inhibitors dalam penanganan hipe...,melindungi nefron yang tersisa dari cedera leb...,teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",menunjukkan hematuria dan proteinuria pada ana...,maka diberikan obat antihipertensi golongan AC...,tingkat yang sesuai dengan toleransi kardiovas...,\nEPO dan hemoglobin.
4,berapa kebutuhan protein harian anak 3-10 tahu...,"4-6 tahun: 1,2 g/kg BB\n7-10 tahun: 1,0 g/kg BB",tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Rekomendasi Nutrisi Bagi Anak Dengan Penyakit ...,. Asupan protein untuk anak PGK berdasarkan DR...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,"\n## Discussion\n\nIn this study, we aimed to ..."
5,berapa dosis EPO pada pasien PGK dengan anemia?,"Dosis biasa EPO adalah 300 unit/kg, dibagi dal...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",heparin konstan 250-2000 unit/jam \n(biasanya ...,sebagai obat antihipertensi lini pertama pada ...,lanjut untuk memastikan kadar hemoglobin tidak...,\npada pasien PGK dengan anemia?
6,bagaimana penanganan dislipidemia pada anak de...,"intervensi gaya hidup, seperti latihan sedang,...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Dislipidemia pada anak PGK umumnya bersifat at...,gula.\nPada anak PGK yang sudah dibatasi asupa...,(5)Dosis maksimal belum ditentukan. Dosis >3mc...,\nkardiovaskular dan dapat mempercepat progres
7,apa kontraindikasi absolut untuk transplantasi...,"keganasan aktif, terutama jika telah bermetast...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Rekomendasi 80:Transplantasi ginjal di Indones...,Penyakit ginjal kronis stadium 5 merupakan ind...,"antara lain kanker yang aktif, HIV positif, in...",\nTidak semua orang dengan penyakit g
8,apa yang bisa dilakukan untuk menangani Hipovo...,pemulihan cairan ekstraselular dengan natrium ...,tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Penanganan gangguan elektrolitHiponatremia (ko...,"b) Makan buah, sayuran, susu dan lemak jenuh y...",Konsentrasi kalium serum yang ideal pada saat ...,\nPertanyaan : Apa yang perlu dilakukan
9,bagaimana pemberian obat vasolidator langsung ...,"Hidralazin, Dosis oral awal 1-2 mg/kg, Dosis o...",tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",5 dan 5D adalah untuk \nmemeriksa kadar vitami...,ACEi dan ARB terutama digunakan pada anak deng...,. Dosis Obat Hipertensi Pada AnakDiuretikaHidr...,\nBagaimana cara pemberian vasolidator langsun...


In [11]:
excel_pertanyaan["tatalaksana anak dan bayi"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa saja indikasi absolut untuk memulai dialis...,"Indikasi absolut meliputi anuria, gangguan ele...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",ABSTRAK\nPenyakit ginjal kronik (PGK) merupaka...,beratnya kurang dari 10 kg. Sebuah studi \nEro...,"dialisis (CKD5d), hampir dua kali lebih banyak...",Tidak semua teks relevans dengan pertanyaan y
2,Apa saja manifestasi klinis yang sering dijump...,Manifestasi klinis meliputi gangguan pertumbuh...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",dominan. \nPenyebab \nutama \nPGK \npada \nana...,PGK pada anak mempunyai karakteristik dan tant...,3)Pemeriksaan eGFR dengan menggunakan formula ...,\nyang mempengaruhi kualitas hidup anak sec
3,Apa perbedaan penting antara penyakit ginjal t...,"Perbedaan penting adalah etiologi PGK, dimana ...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...","KDOQI) pada tahun 2002, merupakan penyakit \ng...",PGK pada anak mempunyai karakteristik dan tant...,3)Pemeriksaan eGFR dengan menggunakan formula ...,\n4)Penentuan Stadium PGK pada anak lebih
4,Apa dampak PGK pada pertumbuhan anak?,Gangguan pertumbuhan yang meningkat saat LFG m...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,4. Kortikosteroid Penggunaan kortikosteroid da...,PGK pada anak mempunyai karakteristik dan tant...,\nPertanyaan : Apa dampak PGK pada pertumb
5,apa yang memengaruhi keberhasilan hemodialisis...,akses vaskular yang baik: fistula arteriovenos...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",kebocoran darah berbunyi atau didapatkan darah...,"lain, ada penurunan risiko hipotensi intra-\nd...",Penyebab disfungsi kateter hemodialisis melipu...,"\nml, pemberian air panas pada catatan kesehat"
6,apa anti koagulan yang paling sering digunakan...,Unfractionated heparin (UFH),teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",setiap kali dialisis harus dapat menghindarkan...,sebaiknya dikerjakan jika dicurigai terdapat i...,"HD), online hemodiafiltration (HDF) dan expan...",\n(2)Dialisis dengan heparin\n(a
7,apa gejala Sindrom disekuilibrium?,"nyeri kepala atau kejang, muntah, kelelahan, m...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",laju \naliran \ndarah \ndipertahankan \npada \...,pasien risiko tinggi (terutama bila urea >60 m...,c) Gejala terkait sistem saraf pusat (nyeri ke...,\nhipokalsemia sebelum dialisis. Untuk
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
with pd.ExcelWriter(f'pertanyaan benchmark_{embed_model}_{loader}_{reranker}.xlsx') as writer:  
    excel_pertanyaan["PNPK"].to_excel(writer, sheet_name='PNPK')
    excel_pertanyaan["Nutrisi"].to_excel(writer, sheet_name='Nutrisi')
    excel_pertanyaan["tatalaksana pada anak"].to_excel(writer, sheet_name='tatalaksana pada anak')
    excel_pertanyaan["tatalaksana anak dan bayi"].to_excel(writer, sheet_name='tatalaksana anak dan bayi')